In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook



X.shape, y.shape

import warnings
warnings.simplefilter("error")
warnings.simplefilter("ignore", DeprecationWarning)

In [37]:
import pandas as pd
import numpy as np


In [38]:
class Node:
    def __init__(self, feature_index, split_value, groups):
        self.feature_index = feature_index
        self.split_value = split_value
        self.groups = groups
        self.left = None
        self.right = None
        
        
    def __str__(self):
        return 'values = ' + str(self.split_value) +', index = '+ str(self.feature_index)

In [39]:
def gini_index(target_groups, classes):
    N = len(target_groups[0]+target_groups[1])
    gini = 0
    for t in target_groups:
        s = 0
        if len(t)==0:
            continue
        for c in classes:
            temp = t.count(c)/len(t)
            temp = temp*temp
            s+=temp
        gini+=(1-s)*(len(t)/N)
    return gini

In [40]:
def generic_split(feature_index, value, Xdataset,Ydataset):
    lX,rX,lY,rY = [],[],[],[]
    
    for i in range(0,len(Xdataset)):
        row = Xdataset[i]
        target = Ydataset[i]
        if row[feature_index] < value:
            lX.append(row)
            lY.append(target)
        else:
            rX.append(row)
            rY.append(target)
            
    return lX,lY,rX,rY

In [71]:
def best_split(Xdataset, Ydataset):
    N = len(Xdataset)
    feature_space = len(Xdataset[0])
    g = np.inf
    v = None
    feature = None
    group = []
    
    for i in range(0,N):
        for j in range(0,feature_space):
            lx,ly,rx,ry = generic_split(j,Xdataset[i][j],Xdataset,Ydataset)
            g_index = gini_index([ly,ry],list(set(Ydataset)))
            print('X%d < %.3f Gini=%.3f' % ((j+1), Xdataset[i][j], g_index))
            if g_index < g:
                g = g_index
                feature = j
                value = Xdataset[i][feature]
                group = [[lx,ly],[rx,ry]]
    result = Node(feature, value, group)
    return result

In [72]:
def to_terminal(targets):
    from collections import Counter
    c = Counter(targ)
    return max(c, key=c.get)

In [73]:
def main_split(root, max_depth, min_size, depth):
    left_X,left_Y = root.groups[0]
    right_X,right_Y = root.groups[1]
    
    if not left_Y or not right_Y:
        root.left = root.right = to_terminal(left_Y + right_Y)
        return

    if depth >= max_depth:
        root.left = to_terminal(left_Y)
        root.right = to_terminal(right_Y)
        return 

    if len(left_Y) <  min_size:
        root.left = to_terminal(left_Y)
    else:
        root.left = best_split(left_X, left_Y)
        main_split(root.left, max_depth, min_size, depth+1)

    if len(right_Y) <  min_size:
        root.right = to_terminal(right_Y)
    else:
        root.right = best_split(right_X, right_Y)
        main_split(root.right, max_depth, min_size, depth+1)

In [74]:
def build_tree(X_train,y_train, max_depth, min_size):
    root = best_split(X_train,y_train)
    main_split(root, max_depth, min_size, 1)
    return root

In [75]:
df = pd.read_csv('banknotes.txt', header = None)
df.columns = ['X_0','X_1','X_2','X_3','Y']
X = df[['X_0','X_1','X_2','X_3']].values
y = df['Y'].values

In [76]:
def predict(model, datapoint):
    value = model.split_value
    index = model.feature_index
    while True:
        if datapoint[index] < value:
            model = model.left
            if isinstance(model,Node):
                value = model.split_value
                index = model.feature_index
            else:
                result = model
                break
        else:
            model = model.right
            if isinstance(model,Node):
                value = model.split_value
                index = model.feature_index
            else:
                result = model
                break
    return result

In [77]:
df = pd.read_csv('banknotes.txt', header = None)
df.columns = ['X_0','X_1','X_2','X_3','Y']

df = df.sample(frac = 1)
X = df[['X_0','X_1','X_2','X_3']].values
y = df['Y'].values

X_train, y_train= X[:100], y[:100]

In [78]:
model = build_tree(X_train, y_train,1000000,1)
np.sum(np.array([predict(model,x) for x in X_train]) == y_train)/len(y_train)

X1 < -3.601 Gini=0.436
X2 < -6.539 Gini=0.448
X3 < 10.523 Gini=0.467
X4 < -0.490 Gini=0.499
X1 < 1.147 Gini=0.281
X2 < 3.599 Gini=0.394
X3 < 1.939 Gini=0.499
X4 < -0.434 Gini=0.499
X1 < 2.957 Gini=0.356
X2 < -4.594 Gini=0.415
X3 < 5.907 Gini=0.456
X4 < 0.572 Gini=0.487
X1 < 0.867 Gini=0.242
X2 < 5.564 Gini=0.383
X3 < 1.677 Gini=0.494
X4 < -0.168 Gini=0.499
X1 < 4.827 Gini=0.475
X2 < 0.803 Gini=0.448
X3 < 1.637 Gini=0.492
X4 < 1.188 Gini=0.491
X1 < -0.015 Gini=0.255
X2 < -1.024 Gini=0.481
X3 < -0.940 Gini=0.496
X4 < 0.650 Gini=0.490
X1 < 3.923 Gini=0.422
X2 < -3.247 Gini=0.463
X3 < 3.458 Gini=0.494
X4 < 0.837 Gini=0.490
X1 < -3.073 Gini=0.415
X2 < -0.532 Gini=0.473
X3 < 2.388 Gini=0.492
X4 < 0.776 Gini=0.484
X1 < -0.706 Gini=0.266
X2 < 8.724 Gini=0.449
X3 < 2.221 Gini=0.500
X4 < -4.596 Gini=0.500
X1 < 2.974 Gini=0.364
X2 < 8.794 Gini=0.443
X3 < -3.636 Gini=0.500
X4 < -1.375 Gini=0.500
X1 < 0.214 Gini=0.225
X2 < -0.695 Gini=0.482
X3 < 0.877 Gini=0.495
X4 < 0.297 Gini=0.498
X1 < -0.122 Gi

X3 < 10.887 Gini=0.000
X4 < -0.528 Gini=0.000
X1 < -4.855 Gini=0.000
X2 < -5.904 Gini=0.000
X3 < 10.982 Gini=0.000
X4 < -0.822 Gini=0.000
X1 < -4.293 Gini=0.000
X2 < 3.342 Gini=0.000
X3 < 0.773 Gini=0.000
X4 < -0.998 Gini=0.000
X1 < -3.993 Gini=0.000
X2 < 5.833 Gini=0.000
X3 < 0.547 Gini=0.000
X4 < -4.938 Gini=0.000
X1 < -4.634 Gini=0.000
X2 < -12.751 Gini=0.000
X3 < 16.717 Gini=0.000
X4 < -3.217 Gini=0.000
X1 < -3.917 Gini=0.000
X2 < 2.665 Gini=0.000
X3 < 0.789 Gini=0.000
X4 < -0.782 Gini=0.000
X1 < -4.505 Gini=0.000
X2 < -5.813 Gini=0.000
X3 < 10.887 Gini=0.000
X4 < -0.528 Gini=0.000
X1 < -4.855 Gini=0.000
X2 < -5.904 Gini=0.000
X3 < 10.982 Gini=0.000
X4 < -0.822 Gini=0.000
X1 < -4.634 Gini=0.000
X2 < -12.751 Gini=0.000
X3 < 16.717 Gini=0.000
X4 < -3.217 Gini=0.000
X1 < -4.505 Gini=0.000
X2 < -5.813 Gini=0.000
X3 < 10.887 Gini=0.000
X4 < -0.528 Gini=0.000
X1 < -4.855 Gini=0.000
X2 < -5.904 Gini=0.000
X3 < 10.982 Gini=0.000
X4 < -0.822 Gini=0.000
X1 < -4.855 Gini=0.000
X2 < -5.904 Gin

0.51